# 학습 데이터셋에서 RGB Mean, Std 추출 후 파일로 저장

In [2]:
import os
import numpy as np
from PIL import Image
from torchvision import transforms

# 학습 데이터셋 경로
train_paths = ['./datasets/images/train', './datasets/images/val']

# 색상 통계 계산을 위한 변수
mean_list = []
std_list = []

# 데이터셋 경로의 모든 이미지를 순회하며 색상 정보 추출
for path in train_paths:
    for img_name in os.listdir(path):
        img_path = os.path.join(path, img_name)
        image = Image.open(img_path).convert("RGB")
        
        # 이미지를 텐서로 변환
        image_tensor = transforms.ToTensor()(image)
        
        # 채널별 평균 및 표준 편차 계산
        mean_list.append(image_tensor.mean(dim=(1, 2)).numpy())
        std_list.append(image_tensor.std(dim=(1, 2)).numpy())

# 전체 이미지의 평균 및 표준 편차 계산
mean = np.mean(mean_list, axis=0)
std = np.mean(std_list, axis=0)

# 색상 정보를 파일로 저장
np.savez("train_color_stats.npz", mean=mean, std=std)
print("색상 정보가 train_color_stats.npz 파일로 저장되었습니다.")


색상 정보가 train_color_stats.npz 파일로 저장되었습니다.


In [4]:
color_stats_file = "train_color_stats.npz"
color_stats = np.load(color_stats_file)
mean = color_stats['mean']
std = color_stats['std']
print("평균:", mean)
print("표준 편차:", std)


평균: [0.18371484 0.18895996 0.19103165]
표준 편차: [0.1316059  0.12825565 0.12362472]


# 저장된 RGB 값 파일 load 후 test dataset에 적용

In [ ]:
import os
from PIL import Image, ImageFile
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

ImageFile.LOAD_TRUNCATED_IMAGES = True

# 이미지 목록을 가져오는 함수
def get_imglist(dir="./sample/img"):
    imglist = [os.path.join(dir, f).replace("\\", "/") for f in os.listdir(dir) if f.endswith('.png')]
    return imglist

class CroppedImageDataset(Dataset):
    def __init__(self, image_list, crop_size, color_stats_file):
        self.image_list = image_list
        self.crop_size = crop_size
        self.transform = transforms.ToTensor()  # 이미지 -> 텐서 변환
        
        # 색상 통계 정보 불러오기
        color_stats = np.load(color_stats_file)
        self.mean = color_stats['mean']
        self.std = color_stats['std']
        self.color_transform = transforms.Normalize(mean=self.mean.tolist(), std=self.std.tolist())

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        image_path = self.image_list[idx]
        image_name = os.path.basename(image_path)

        # 이미지 열기
        image = Image.open(image_path)
        image_width, image_height = image.size

        # 전체 크롭 이미지 개수 계산
        num_crops_x = (image_width + self.crop_size - 1) // self.crop_size
        num_crops_y = (image_height + self.crop_size - 1) // self.crop_size
        total_crops = num_crops_x * num_crops_y

        # 크롭할 영역의 좌상단 좌표를 슬라이딩 윈도우 방식으로 구함
        cropped_images = []
        positions = []
        last_cropped_image_info = None  # 마지막 크롭된 이미지 정보 저장

        for top_left_x in range(0, image_width, self.crop_size):
            for top_left_y in range(0, image_height, self.crop_size):
                # 마지막 부분에서 경계 넘지 않도록 마지막 부분을 맞춤
                bottom_right_x = min(top_left_x + self.crop_size, image_width)
                bottom_right_y = min(top_left_y + self.crop_size, image_height)

                # 이미지 경계 부분에 대해 크롭 영역을 이동시킴
                if bottom_right_x - top_left_x < self.crop_size:
                    top_left_x = image_width - self.crop_size
                    bottom_right_x = image_width

                if bottom_right_y - top_left_y < self.crop_size:
                    top_left_y = image_height - self.crop_size
                    bottom_right_y = image_height

                # 크롭한 이미지 자르기
                cropped_image = image.crop((top_left_x, top_left_y, bottom_right_x, bottom_right_y))

                # 크롭한 이미지를 텐서로 변환
                cropped_image_tensor = self.transform(cropped_image)
                normalized_image_tensor = self.color_transform(cropped_image_tensor)

                # 크롭한 이미지와 좌상단 좌표 저장
                cropped_images.append(normalized_image_tensor)
                positions.append(torch.tensor([top_left_x, top_left_y]))

                # 마지막 크롭된 이미지의 정보 저장 (좌표와 실제 크기)
                last_cropped_image_info = {
                    'image_tensor': normalized_image_tensor,
                    'top_left': (top_left_x, top_left_y),
                    'bottom_right': (bottom_right_x, bottom_right_y),
                    'size': (bottom_right_x - top_left_x, bottom_right_y - top_left_y)  # 실제 크기 저장
                }

        # 이미지 이름, 크롭한 이미지 텐서 목록, 각 이미지의 좌상단 좌표 및 크롭 개수 반환
        return {
            'image_name': image_name,
            'images': cropped_images,  # 잘라낸 이미지 텐서 리스트
            'top_left_positions': positions,  # 각 이미지의 좌상단 좌표 리스트
            'total_crops': total_crops,  # 총 크롭 이미지 개수
            'last_cropped_image_info': last_cropped_image_info  # 마지막 크롭 이미지 정보
        }

# 배치 데이터를 처리하는 collate_fn 정의
def collate_fn(batch, batch_size):
    all_image_names = []
    all_images = []
    all_top_left_positions = []
    total_crops = 0  # 전체 크롭 이미지 개수를 추적
    last_cropped_images_info = []  # 마지막 크롭 이미지 정보 추적

    for item in batch:
        image_names = [item['image_name']] * len(item['images'])  # 각 이미지에 같은 이름을 붙임
        all_image_names.extend(image_names)
        all_images.extend(item['images'])  # 이미지를 리스트에 추가
        all_top_left_positions.extend(item['top_left_positions'])  # 좌상단 좌표 추가
        total_crops += item['total_crops']  # 총 크롭 개수 계산
        last_cropped_images_info.append(item['last_cropped_image_info'])  # 마지막 크롭 정보 추가

    # 전체 이미지 목록을 batch_size 크기씩 나눠서 반환
    batch_start = 0
    while batch_start < len(all_images):
        images_batch = torch.stack(all_images[batch_start:batch_start + batch_size])  # batch_size만큼 이미지 묶기
        positions_batch = torch.stack(all_top_left_positions[batch_start:batch_start + batch_size])  # batch_size만큼 좌표 묶기
        names_batch = all_image_names[batch_start:batch_start + batch_size]  # batch_size만큼 이미지 이름 묶기
        
        batch_start += batch_size
        
        yield {
            'image_names': names_batch,  # 이미지 이름 리스트
            'images': images_batch,  # [batch_size, 3, crop_size, crop_size]
            'top_left_positions': positions_batch,  # [batch_size, 2]
            'total_crops': total_crops,  # 전체 크롭 이미지 개수
            'last_cropped_images_info': last_cropped_images_info  # 마지막 크롭 이미지 정보
        }

# 사용 예시
directory_path = '/workspace/dataset'
crop_size = 1024  # 크롭할 이미지의 크기
img_list = get_imglist(directory_path)